# Imports

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from typing import List, Tuple
from transformers import AutoTokenizer

load_dotenv()

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

model = "gpt-4o-mini"

In [2]:
file_path = '/Users/renatoboemer/code/developer/luthor/data/Memo 2  - Crypto assets disposal - FINISHED .docx'

# Data processing

## Load

In [3]:
import os
import fitz
from docx import Document

def read_file(file_path):
    # Check if file exists
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"The file {file_path} does not exist.")
    
    # Get the file extension
    _, file_extension = os.path.splitext(file_path)

    # Handle .txt files
    if file_extension.lower() == '.txt':
        return read_txt(file_path)

    # Handle .docx files
    elif file_extension.lower() == '.docx':
        return read_docx(file_path)

    # Handle .pdf files
    elif file_extension.lower() == '.pdf':
        return read_pdf(file_path)

    else:
        raise ValueError(f"Unsupported file extension: {file_extension}")

In [4]:
def read_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def read_docx(file_path):
    document = Document(file_path)
    full_text = []
    for paragraph in document.paragraphs:
        full_text.append(paragraph.text)
    return '\n'.join(full_text)


def read_pdf(file_path):
    document = fitz.open(file_path)
    all_text = []
    for page in document:
        text = page.get_text()
        all_text.append(text)
    return '\n'.join(all_text)

In [5]:
text = read_file(file_path)
print(text[:350])

Input:
I would like to seek expert advice regarding our company's upcoming transaction involving the disposal of cryptocurrency assets. Specifically, we are planning to sell a substantial amount of Ethereum (ETH) and convert it into stablecoins due to market volatility. Please highlight the tax implications of that. Do we need to formalise the sale


## Process

In [6]:
import re

def text_segmentation(text: str) -> List[str]:
    """
    Split text into smaller, manageable chunks (e.g., paragraphs).

    Args:
        text (str): The full text to be segmented.

    Returns:
        List[str]: A list of segmented text chunks.
    """
    # Split text by double newlines or similar paragraph markers
    segments = re.split(r'\n\s*\n', text.strip())
    
    return [segment.strip() for segment in segments if segment.strip()]

In [7]:
def tokenize_text(text: str) -> List[str]:
    """
    Tokenize the text into words.

    Args:
        text (str): The text to be tokenized.

    Returns:
        List[str]: A list of tokens (words).
    """
    # simple regex to split words
    tokens = re.findall(r'\b\w+\b', text)
    
    return tokens

In [8]:
def clean_special_characters(text: str) -> str:
    """
    Clean up non-informative special characters or artifacts.

    Args:
        text (str): The text from which to remove special characters.

    Returns:
        str: Cleaned text with unnecessary special characters removed.
    """
    # Remove characters not usually found in legal texts
    cleaned_text = re.sub(r'[^\w\s,.!?;:()-]', '', text)
    
    return cleaned_text

In [9]:
def preserve_structure(text: str) -> str:
    """
    Maintain the document's structural elements, such as headings.

    Args:
        text (str): The text to process for structural preservation.

    Returns:
        str: Text with preserved structure for headings and sections.
    """
    # Keep lines starting with capital words as headings
    structured_text = re.sub(r'(?m)^(?=[A-Z])(.+)$', r'## \1', text)
    
    return structured_text

In [44]:
def create_chunks(text: str, tokenizer, chunk_size=4096) -> List[str]:
    """
    Creates chunks of text for preprocessing, ensuring each chunk is within the specified size.

    Args:
        text (str): The full text to be chunked.
        tokenizer (object): The tokenizer object to encode and decode text.
        chunk_size (int): The desired size of each chunk.

    Returns:
        List[str]: A list of text chunks.
    """
    # Tokenize the text
    tokens = tokenizer.encode(text, add_special_tokens=True, truncation=True, max_length=chunk_size)

    # Split tokens into chunks of the specified size
    chunks = [tokens[i:i + chunk_size] for i in range(0, len(tokens), chunk_size)]

    # Decode each chunk back into text
    text_chunks = [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]

    return text_chunks


In [45]:
def preprocess_doc(file_path: str, tokenizer, chunk_size=4096, overlap=0) -> Tuple[str, List[str], str, List[str], str, List[str]]:
    """
    Preprocess a legal document by executing a series of text processing steps, including chunking.

    Args:
        file_path (str): The path to the legal document text file.
        tokenizer (object): The tokenizer object to encode and decode text.
        chunk_size (int): The desired size of each chunk.
        overlap (int): The number of tokens to overlap between chunks.

    Returns:
        Tuple: A tuple containing:
            - Original text (str)
            - Segmented text chunks (List[str])
            - Cleaned text (str)
            - Tokenized words (List[str])
            - Structured text (str)
            - Chunks (List[str])
    """
    # Load text from the file
    text = read_file(file_path)

    # Split text into segments (paragraphs)
    segments = text_segmentation(text)

    # Clean up non-informative special characters
    cleaned_text = clean_special_characters(text)

    # Tokenize the text into words
    tokens = tokenize_text(cleaned_text)

    # Preserve structural elements, e.g., headings
    structured_text = preserve_structure(cleaned_text)

    # Create chunks
    chunks = create_chunks(cleaned_text, tokenizer, chunk_size)

    return text, segments, cleaned_text, tokens, structured_text, chunks
    

In [46]:
# Using the preprocessor
from transformers import LongformerTokenizer

# Initialise the Longformer tokenizer
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')

# Call the preprocess_doc function
original_text, segments, cleaned_text, tokens, structured_text, chunks = preprocess_doc(file_path, tokenizer)

print("Original Text:", original_text[:50], "...")
print("---" * 25)
print("Segments:", segments[:1])
print("---" * 25)
print("Cleaned Text:", cleaned_text[:50], "...")
print("---" * 25)
print("Tokens:", tokens[:5])
print("---" * 25)
print("Structured Text:", structured_text[:50], "...")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

Original Text: Input:
I would like to seek expert advice regardin ...
---------------------------------------------------------------------------
Segments: ["Input:\nI would like to seek expert advice regarding our company's upcoming transaction involving the disposal of cryptocurrency assets. Specifically, we are planning to sell a substantial amount of Ethereum (ETH) and convert it into stablecoins due to market volatility. Please highlight the tax implications of that. Do we need to formalise the sale of ETH with a formal agreement?\nFollow up question: What is the volume of the transactions?\nFollow up question: What is the date of the transactions\nFollow up question: What was the price of purchased ETH?/ What was the value of ETH expressed in stablecoin?\nFollow up question: What is the stablecoin pegged to? \nFollow up question: Has this transaction will happen between dependent or independent parties?\nFollow up question: When is the transaction planned for?\nFollow up question

In [67]:
# Tokenize the cleaned_text to inspect the embedding size
sample_tokenized = tokenizer(cleaned_text, return_tensors="pt", max_length=4096, truncation=True)

# Print the size of the embedding
embedding_size = sample_tokenized['input_ids'].size(1)  # Get the number of tokens
print(f"Embedding size: {embedding_size}")

Embedding size: 1517


## Embedding

In [48]:
# openai embedding function
def get_embedding(docs: list[str]) -> list[list[float]]:
    res = client.embeddings.create(
        input=docs,
        model="text-embedding-3-small"
    )
    
    doc_embeds = [r.embedding for r in res.data]
    
    return doc_embeds


In [50]:
# Generate embeddings for the segmented text chunks
chunk_embeddings = get_embedding(chunks)

## Query the System

In [105]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import openai
from pinecone import Pinecone, ServerlessSpec

# Load environment variables from a .env file
load_dotenv()

class RAGSystem:
    def __init__(self, index_name: str, pinecone_api_key: str, openai_api_key: str, embedding_model: str="text-embedding-ada-002"):
        self.index_name = index_name
        self.embedding_model = embedding_model

        # Initialize Pinecone and OpenAI clients
        self.pinecone_client = self.initialize_pinecone(pinecone_api_key)
        self.openai_client = self.initialize_openai(openai_api_key)
        self.index = self.pinecone_client.Index(self.index_name)

    def initialize_pinecone(self, api_key: str):
        # Create a Pinecone client instance
        pc = Pinecone(api_key=api_key)

        # Check if the index exists; if not, create it
        if self.index_name not in pc.list_indexes().names():
            pc.create_index(
                name=self.index_name,
                dimension=1536,  # Ensure this matches the output size of the embedding model
                metric='cosine',
                spec=ServerlessSpec(
                    cloud='aws',
                    region='us-east-1'
                )
            )
        return pc

    def initialize_openai(self, api_key: str):
        # Initialise OpenAI client
        openai = OpenAI(
            api_key=api_key,
        )
        return openai

    def get_index(self):
        # Retrieve or create the index from Pinecone
        return self.pinecone_client.Index(self.index_name)
    
    def get_embedding(self, text: str):
        # Use OpenAI API to get text embeddings
        response = self.openai_client.embeddings.create(input=text, model=self.embedding_model)
        return response.data[0].embedding

    def upsert_vectors(self, vectors: list):
        # Upsert vectors using the Pinecone index
        try:
            index = self.get_index()
            index.upsert(vectors=vectors)
            print(f"Successfully upserted {len(vectors)} vectors to index {self.index_name}.")
        except Exception as e:
            print(f"Upsert Error: {e}")

    def query_vectors(self, query_vector: list, top_k: int = 3):
        # Query the Pinecone index
        index = self.get_index()
        result = index.query(vector=query_vector, top_k=top_k)
        print("Query results:", result)  # Debugging print
        return result['matches']

    def retrieve_documents(self, question: str, top_k: int = 3):
        # Retrieve documents relevant to the question
        query_embedding = self.get_embedding(question)
        matches = self.query_vectors(query_embedding, top_k=top_k)
        print("Query matches:", matches)  # Debugging print
        # Check the actual keys available in matches to adjust retrieval
        return [match.metadata.text for match in matches if 'metadata' in match and 'text' in match.metadata]

    def generate_answer(self, question: str, context: str, model: str='gpt-4o-mini'):
        # Generate an answer using OpenAI API
        prompt = f"""You are a lawyer that helps retrieve knowledge from past
                     memos and documents to answer subsequent questions.
                     If the answer cannot be found, write "I don't know."

        Context: {context}

        Question: {question}"""

        response = self.openai_client.Completion.create(
            engine=model,
            prompt=prompt,
            max_tokens=50,
            temperature=0
        )

        return response.choices[0].text.strip()

    def answer_question(self, question: str, top_k: int=3):
        # Provide an answer to the question
        documents = self.retrieve_documents(question, top_k)
        context = "\n".join(documents)
        if not context:
            return "I don't know."
        return self.generate_answer(question, context)


In [106]:
# Define the index name and API keys
index_name = 'luthor-test-nb-0'
pinecone_api_key = os.getenv('PINECONE_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')

# Assuming the following variables are defined:
# - file_path: path to the document file
# - tokenizer: Longformer tokenizer
# - preprocess_doc: function that preprocesses the document
original_text, segments, cleaned_text, tokens, structured_text, chunks = preprocess_doc(file_path, tokenizer)

# Initialize the RAG system
rag_system = RAGSystem(index_name, pinecone_api_key, openai_api_key)

# Generate embeddings for the segmented text chunks
chunk_embeddings = [rag_system.get_embedding(chunk) for chunk in chunks]

# Create vector dictionaries and upsert documents
vectors = [
    {"id": f"doc_{i}", "values": chunk_embeddings[i], "metadata": {"text": chunks[i]}}
    for i in range(len(chunks))
]

rag_system.upsert_vectors(vectors)

# Answer a question using the RAG system
question = "What are the essential elements of a contract in English law?"
answer = rag_system.answer_question(question)
print(answer)

Successfully upserted 1 vectors to index luthor-test-nb-0.
Query results: {'matches': [{'id': 'doc_0', 'score': 0.75419426, 'values': []},
             {'id': 'doc0', 'score': 0.00510411849, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}
Query matches: [{'id': 'doc_0', 'score': 0.75419426, 'values': []}, {'id': 'doc0', 'score': 0.00510411849, 'values': []}]
I don't know.
